In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

In [3]:
import matplotlib.image as mpimg
import numpy as np
import skimage
import dlib
import numpy as np
from PIL import Image
import glob
import cv2
import os
import torchvision.transforms.functional as tf
import random


In [4]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# file_name_list = glob.glob('../train_org_img/*/*')
file_name_list = glob.glob('../train_org_img/SURPRISE/*')
print(file_name_list)

['../train_org_img/ANGRY/angry1.jpg', '../train_org_img/ANGRY/angry10.jpg', '../train_org_img/ANGRY/angry11.jpg', '../train_org_img/ANGRY/angry12.jpg', '../train_org_img/ANGRY/angry13.jpg', '../train_org_img/ANGRY/angry15.jpg', '../train_org_img/ANGRY/angry16.jpg', '../train_org_img/ANGRY/angry17.jpg', '../train_org_img/ANGRY/angry18.jpg', '../train_org_img/ANGRY/angry19.jpg', '../train_org_img/ANGRY/angry2.jpg', '../train_org_img/ANGRY/angry20.jpg', '../train_org_img/ANGRY/angry21.jpg', '../train_org_img/ANGRY/angry23.jpg', '../train_org_img/ANGRY/angry24.jpg', '../train_org_img/ANGRY/angry25.jpg', '../train_org_img/ANGRY/angry26.jpg', '../train_org_img/ANGRY/angry27.jpg', '../train_org_img/ANGRY/angry28.jpg', '../train_org_img/ANGRY/angry29.jpg', '../train_org_img/ANGRY/angry3.jpg', '../train_org_img/ANGRY/angry30.jpg', '../train_org_img/ANGRY/angry4.jpg', '../train_org_img/ANGRY/angry5.jpg', '../train_org_img/ANGRY/angry6.jpg', '../train_org_img/ANGRY/angry7.jpg', '../train_org_img/

In [5]:
# load bgimg:
bg_list = glob.glob('bgimg/*')

def rand_load_bgimg():
    bg_idx = random.randint(0,4)
#     bg_list.length()
    bg_path = bg_list[bg_idx]
#     bg_img = dlib.load_rgb_image(bg_path)
    bg_img = Image.open(bg_path)
#     plt.imshow(bg_img)
    area = (0, 0, 256, 256)
    bg_img = bg_img.crop(area)
    bg_img = tf.to_grayscale(bg_img, num_output_channels=3)
    return bg_img

# rand_load_bgimg()


In [6]:
def transFace(face_with_mask):
#     eval_transformer = torchvision.transforms.Compose([
#     torchvision.transforms.RandomAffine((-180, 180), translate=(0.5,0.5), scale=(0.5,1.5))
#     torchvision.transforms.ToTensor(),
#     transforms.Normalize((0.485, 0.456, 0.406),
#                          (0.229, 0.224, 0.225)),
#     ])  # transform it into a torch tensor

    pil_with_mask = Image.fromarray(face_with_mask)
    angle = random.randint(-180, 180)
    scale = random.uniform(0.3,1.5)
    translateH = random.randint(-20, 20)
    translateV = random.randint(-20, 20)

    trans_img = tf.affine(img = pil_with_mask, angle = angle, translate = (translateH, translateV), scale = scale, shear = 0)

#     torchvision.transforms.RandomAffine(degrees, translate=None, scale=None, shear=None, resample=False, fillcolor=0)
    return trans_img


In [7]:
def appendFace(trans_pil):
    bg_img_pil = rand_load_bgimg()
    trans_np = np.array(trans_pil)
    bg_img_np = np.array(bg_img_pil)
    bg_img_np = np.reshape(bg_img_np,(256,256,3))
    face_mask = trans_np[:,:,2]
    face_mask_3_dim = np.reshape(face_mask,(256,256,1))
    bg_img_np_mask = bg_img_np * (1-face_mask_3_dim)
    appended_img = bg_img_np_mask + trans_np
#     bg_img_pil.paste(trans_pil)
#     bg_img_pil.show()
    return appended_img

In [8]:
def getMaskedImage(file_path):
    img = dlib.load_rgb_image(file_path)
#     print(file_path)

    emotion_tag = file_path.split("/")[2]
    filename = file_path.split("/")[3].split(".")[0]
    
    rect = detector(img)[0]
    sp = predictor(img, rect)
    landmarks = np.array([[p.x, p.y] for p in sp.parts()])
    outline = landmarks[[*range(17), *range(26,16,-1)]]
    Y, X = skimage.draw.polygon(outline[:,1], outline[:,0])
    np.clip(X, 0, 255, out = X)
    np.clip(Y, 0, 255, out = Y)
#     clip for out range
    cropped_img = np.zeros(img.shape, dtype=np.uint8)
    cropped_img[Y, X] = img[Y, X]
    
    img_mask = np.zeros(img.shape,dtype=np.uint8)
    img_mask[Y,X] = 1
#     call trans
# SAVE TRANS IMG
    # TODO?: save mask as [:,:,3] here?
    cropped_img[:,:,2] = img_mask[:,:,2]
    
    counter = 0
    for counter in range(5):
        trans_img_pil = transFace(cropped_img)
    #     Image.fromarray(trans_img).save(outimg_path)
        pil_comb_np = appendFace(trans_img_pil)
        pil_comb_pil = Image.fromarray(pil_comb_np)
        br_ct = 0
        
        for br_ct in range(4):
            brightness_factor = random.uniform(0,2)
            result_img = tf.adjust_brightness(pil_comb_pil, brightness_factor)
    #         Image.fromarray(pil_comb).save(outimg_path)
            outimg_path = os.path.join("outimg", emotion_tag, filename + "_f" + str(counter) + "_br" + str(br_ct)+ ".jpg" )
            result_img.save(outimg_path)
            br_ct +=1
            
        counter += 1
#     trans_img.save(pil_img)
#     plt.imshow(img_mask[:,:,2])
#     img_mask.shape
#     return cropped_img

In [9]:
counter = 0
for file_path in file_name_list:
    getMaskedImage(file_path)
#     print (file_path)
#     croppedImg = getMaskedImage(filePath)
# plt.imshow(croppedImg)

IndexError: list index out of range

In [10]:
Image.fromarray(np.zeros(3,3,3))

TypeError: data type not understood

In [12]:
Image.fromarray(np.zeros(100,100,3))

TypeError: data type not understood